In [21]:
import numpy as np
import random
import itertools

# Constants
MAGENTA = 'M'
YELLOW = 'Y'

# Global counters & records
individual_id_counter = 1
genetic_data_records = []
chromatid_recombination_records = []

# Classes
class Chromosome:
    def __init__(self, alleles):
        self.alleles = alleles

    def __repr__(self):
        snippet = ''.join(self.alleles[:10]) if self.alleles else ''
        return f"Chr({snippet}...)"

class DiploidChromosomePair:
    def __init__(self, chromatid1, chromatid2):
        self.chromatid1 = chromatid1
        self.chromatid2 = chromatid2

    def __repr__(self):
        return f"Pair(\n  {self.chromatid1}\n  {self.chromatid2}\n)"

class Individual:
    def __init__(self, num_chromosomes, num_loci_per_chromosome):
        global individual_id_counter
        self.id = individual_id_counter
        individual_id_counter += 1

        self.num_chromosomes = num_chromosomes
        self.num_loci_per_chromosome = num_loci_per_chromosome
        self.diploid_chromosome_pairs = []

    def __repr__(self):
        return f"Individual(ID: {self.id}, Chromosomes: {len(self.diploid_chromosome_pairs)})"

    def get_all_numeric_genotypes(self):
        all_numeric = []
        for pair in self.diploid_chromosome_pairs:
            a1 = pair.chromatid1.alleles
            a2 = pair.chromatid2.alleles
            for i in range(self.num_loci_per_chromosome):
                pair_sorted = sorted([a1[i], a2[i]])
                if pair_sorted == [MAGENTA, MAGENTA]:
                    all_numeric.append(2)
                elif pair_sorted == [YELLOW, YELLOW]:
                    all_numeric.append(0)
                else:
                    all_numeric.append(1)
        return all_numeric

    def calculate_hybrid_index(self):
        all_numeric = self.get_all_numeric_genotypes()
        total_alleles = len(all_numeric) * 2
        if total_alleles == 0:
            return 0.0
        sum_m = sum(all_numeric)
        return sum_m / total_alleles

    def calculate_heterozygosity(self):
        all_numeric = self.get_all_numeric_genotypes()
        if len(all_numeric) == 0:
            return 0.0
        return all_numeric.count(1) / len(all_numeric)

    def get_chromatid_block_data(self):
        results = []
        for chr_idx, pair in enumerate(self.diploid_chromosome_pairs):
            for chromatid_idx, chromatid in enumerate([pair.chromatid1, pair.chromatid2]):
                junctions, lengths, alleles = self._analyse_single_chromatid(chromatid.alleles)
                results.append({
                    'individual_id': self.id,
                    'diploid_chr_id': chr_idx + 1,
                    'chromatid_in_pair': 'A' if chromatid_idx == 0 else 'B',
                    'total_junctions': junctions,
                    'block_lengths': lengths,
                    'block_alleles': alleles
                })
        return results

    def _analyse_single_chromatid(self, alleles):
        if not alleles:
            return 0, [], []
        block_lengths = []
        block_alleles = []
        for allele, group in itertools.groupby(alleles):
            group_list = list(group)
            block_lengths.append(len(group_list))
            block_alleles.append(allele)
        junctions = len(block_lengths) - 1 if block_lengths else 0
        return junctions, block_lengths, block_alleles

# Recording functions
def record_individual_genome(individual, generation_label):
    for chr_idx, pair in enumerate(individual.diploid_chromosome_pairs):
        for locus_idx in range(individual.num_loci_per_chromosome):
            allele_a = pair.chromatid1.alleles[locus_idx]
            allele_b = pair.chromatid2.alleles[locus_idx]
            genotype_str = f"{allele_a}|{allele_b}"
            genetic_data_records.append({
                'generation': generation_label,
                'individual_id': individual.id,
                'diploid_chr_id': chr_idx + 1,
                'locus_position': locus_idx,
                'genotype': genotype_str
            })

def record_chromatid_recombination(individual, generation_label):
    chromatid_data = individual.get_chromatid_block_data()
    for record in chromatid_data:
        record['generation'] = generation_label
        chromatid_recombination_records.append(record)

# Recombination simulation
def meiosis_with_recombination(diploid_pair, recomb_event_probabilities, recomb_probabilities):
    loci = diploid_pair.chromatid1.alleles
    loci_len = len(loci)

    n_events = random.choices(
        population=[0,1,2],
        weights=[recomb_event_probabilities.get(i,0) for i in [0,1,2]],
        k=1
    )[0]

    possible_positions = list(range(1, loci_len))
    if n_events > 0:
        weights = recomb_probabilities[1:loci_len]
        weights_sum = sum(weights)
        if weights_sum == 0:
            chosen_positions = sorted(random.sample(possible_positions, n_events))
        else:
            norm_weights = [w/weights_sum for w in weights]
            chosen_positions = sorted(random.choices(possible_positions, weights=norm_weights, k=n_events))
    else:
        chosen_positions = []

    parent1 = diploid_pair.chromatid1.alleles
    parent2 = diploid_pair.chromatid2.alleles
    recombinant_alleles = []
    last_pos = 0
    source = 0
    breakpoints = chosen_positions + [loci_len]

    for pos in breakpoints:
        if source == 0:
            recombinant_alleles.extend(parent1[last_pos:pos])
        else:
            recombinant_alleles.extend(parent2[last_pos:pos])
        source = 1 - source
        last_pos = pos

    return Chromosome(recombinant_alleles)

# Cross simulation
def run_genetic_cross(parents_pop_A, parents_pop_B, num_offspring_to_create,
                      generation_label, num_chromosomes_for_offspring,
                      recomb_event_probabilities, recomb_probabilities):
    offspring = []
    for _ in range(num_offspring_to_create):
        parent_A = random.choice(parents_pop_A)
        parent_B = random.choice(parents_pop_B)
        child = Individual(num_chromosomes_for_offspring, parent_A.num_loci_per_chromosome)

        for chr_idx in range(num_chromosomes_for_offspring):
            diploid_pair_A = parent_A.diploid_chromosome_pairs[chr_idx]
            diploid_pair_B = parent_B.diploid_chromosome_pairs[chr_idx]

            haploid_from_A = meiosis_with_recombination(diploid_pair_A, recomb_event_probabilities, recomb_probabilities)
            haploid_from_B = meiosis_with_recombination(diploid_pair_B, recomb_event_probabilities, recomb_probabilities)

            child.diploid_chromosome_pairs.append(DiploidChromosomePair(haploid_from_A, haploid_from_B))

        record_individual_genome(child, generation_label)
        record_chromatid_recombination(child, generation_label)
        offspring.append(child)

    return offspring

# Create pure individuals and populations
def create_pure_individual(num_chromosomes, num_loci_per_chr, allele_type):
    individual = Individual(num_chromosomes, num_loci_per_chr)
    for _ in range(num_chromosomes):
        chr_alleles = [allele_type] * num_loci_per_chr
        chromatid1 = Chromosome(chr_alleles[:])
        chromatid2 = Chromosome(chr_alleles[:])
        individual.diploid_chromosome_pairs.append(DiploidChromosomePair(chromatid1, chromatid2))
    return individual

def create_pure_population(num_individuals, num_chromosomes, num_loci_per_chr, allele_type):
    return [create_pure_individual(num_chromosomes, num_loci_per_chr, allele_type) for _ in range(num_individuals)]

# Create F1 population from two pure populations (assuming equal sizes)
def create_F1_population(pure_pop_A, pure_pop_B):
    if len(pure_pop_A) != len(pure_pop_B):
        raise ValueError("Pure populations must be same size to create F1 population")

    f1_population = []
    for i in range(len(pure_pop_A)):
        parent_A = pure_pop_A[i]
        parent_B = pure_pop_B[i]

        child = Individual(parent_A.num_chromosomes, parent_A.num_loci_per_chromosome)
        for chr_idx in range(parent_A.num_chromosomes):
            chr_A = parent_A.diploid_chromosome_pairs[chr_idx]
            chr_B = parent_B.diploid_chromosome_pairs[chr_idx]

            # Haploid from parent_A: take one chromatid (assumed homozygous)
            haploid_A = chr_A.chromatid1
            # Haploid from parent_B: take one chromatid (assumed homozygous)
            haploid_B = chr_B.chromatid1

            child.diploid_chromosome_pairs.append(DiploidChromosomePair(haploid_A, haploid_B))
        f1_population.append(child)
    return f1_population

# Helper to calculate population stats for reporting
def population_stats(pop):
    his = [ind.calculate_hybrid_index() for ind in pop]
    hets = [ind.calculate_heterozygosity() for ind in pop]
    return {
        'mean_HI': np.mean(his) if his else 0,
        'std_HI': np.std(his) if his else 0,
        'mean_HET': np.mean(hets) if hets else 0,
        'std_HET': np.std(hets) if hets else 0,
        'count': len(pop)
    }

# Unified generation simulation pipeline
def simulate_generations(
    initial_pop_A,
    initial_pop_B,
    generation_plan,
    num_offspring_per_cross,
    num_chromosomes,
    recomb_event_probabilities,
    recomb_probabilities,
    verbose=True
):
    populations = {
        'P_A': initial_pop_A,
        'P_B': initial_pop_B
    }

    # Create F1 population explicitly first if requested
    if ('F1',) in generation_plan:
        f1_pop = create_F1_population(populations['P_A'], populations['P_B'])
        populations['F1'] = f1_pop
        if verbose:
            stats = population_stats(f1_pop)
            print(f"F1 created with {len(f1_pop)} individuals | Mean HI: {stats['mean_HI']:.3f} (±{stats['std_HI']:.3f}), Mean HET: {stats['mean_HET']:.3f} (±{stats['std_HET']:.3f})")

    # Process subsequent generations based on generation_plan
    for gen_info in generation_plan:
        if gen_info == ('F1',):
            continue  # already done above

        gen_name = gen_info[0]
        parents_names = gen_info[1:]

        # Validate parents exist
        for p in parents_names:
            if p not in populations:
                raise ValueError(f"Parent population '{p}' not found for generation '{gen_name}'")

        # Cross parents to create new generation
        parents_pop_A = populations[parents_names[0]]
        parents_pop_B = populations[parents_names[1]]

        new_pop = run_genetic_cross(
            parents_pop_A,
            parents_pop_B,
            num_offspring_per_cross,
            gen_name,
            num_chromosomes,
            recomb_event_probabilities,
            recomb_probabilities
        )

        populations[gen_name] = new_pop
        if verbose:
            stats = population_stats(new_pop)
            print(f"{gen_name} created with {len(new_pop)} individuals from parents {parents_names} | Mean HI: {stats['mean_HI']:.3f} (±{stats['std_HI']:.3f}), Mean HET: {stats['mean_HET']:.3f} (±{stats['std_HET']:.3f})")

    return populations

# Parameters and example usage

num_chromosomes = 2
num_loci_per_chr = 100

# Recombination event probabilities (example: 0 events = 10%, 1 event = 85%, 2 events = 5%)
recomb_event_probabilities = {0: 0.10, 1: 0.85, 2: 0.05}

# Recombination probability along chromosome loci (U-shaped example: higher at ends)
recomb_probabilities = [0] + [0.9 if i == 1 or i == num_loci_per_chr - 1 else 0.1 for i in range(1, num_loci_per_chr)]

# Create pure populations
num_pure_per_pop = 10
pure_pop_A = create_pure_population(num_pure_per_pop, num_chromosomes, num_loci_per_chr, MAGENTA)
pure_pop_B = create_pure_population(num_pure_per_pop, num_chromosomes, num_loci_per_chr, YELLOW)

# Record pure populations
for ind in pure_pop_A:
    record_individual_genome(ind, 'P_A')
    record_chromatid_recombination(ind, 'P_A')

for ind in pure_pop_B:
    record_individual_genome(ind, 'P_B')
    record_chromatid_recombination(ind, 'P_B')

# Define generation plan as tuples:
# Each tuple: (generation_name, parent_pop_1, parent_pop_2)
# For example: ('F2', 'F1', 'F1'), ('BC1', 'F2', 'P_A')
generation_plan = [
    ('F1',),          # create F1 from P_A x P_B (done separately)
    ('F2', 'F1', 'F1'),
    ('BC1', 'F2', 'P_A'),
    ('BC2', 'BC1', 'P_B')
]

num_offspring_per_cross = 20

# Run simulation
populations = simulate_generations(
    pure_pop_A,
    pure_pop_B,
    generation_plan,
    num_offspring_per_cross,
    num_chromosomes,
    recomb_event_probabilities,
    recomb_probabilities,
    verbose=True
)

# Example: Print population sizes summary
print("\nSummary of population sizes:")
for gen, pop in populations.items():
    print(f"{gen} population size: {len(pop)}")


F1 created with 10 individuals | Mean HI: 0.500 (±0.000), Mean HET: 1.000 (±0.000)
F2 created with 20 individuals from parents ('F1', 'F1') | Mean HI: 0.460 (±0.178), Mean HET: 0.385 (±0.162)
BC1 created with 20 individuals from parents ('F2', 'P_A') | Mean HI: 0.758 (±0.123), Mean HET: 0.484 (±0.246)
BC2 created with 20 individuals from parents ('BC1', 'P_B') | Mean HI: 0.402 (±0.081), Mean HET: 0.803 (±0.162)

Summary of population sizes:
P_A population size: 10
P_B population size: 10
F1 population size: 10
F2 population size: 20
BC1 population size: 20
BC2 population size: 20
